# SEIRX model runs for test technology and screening frequency investigation

In [1]:
import networkx as nx
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
from scseirx import analysis_functions as af # custom analysis functions
import data_creation_functions_vaccination as dcf

from importlib import reload

## Parameters

### Simulation parameters and measures in place

In [2]:
with open('params/vaccination_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/vaccination_measures.json') as json_file:
    measures = json.load(json_file)


### Screening parameters

In [3]:
e_vaccination_probabilities = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
r_vaccination_probabilities = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

screening_params = [(i, j, k)\
              for i in index_cases \
              for j in e_vaccination_probabilities \
              for k in r_vaccination_probabilities]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 242 parameter combinations


In [5]:
def run(param_list, N_runs=1000):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # paths for data I/O
    contact_network_src = '../data/contact_networks'

    # extract the simulation parameters from the parameter list
    index_case, e_vaccination_probability, r_vaccination_probability = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, index_case, 
                e_vaccination_probability, r_vaccination_probability, measures,
                simulation_params, contact_network_src)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case,
                        e_vaccination_probability, r_vaccination_probability)

    return row

In [7]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['index_case',
              'resident_vaccination_probability', 'employee_vaccination_probability']
              #'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_vaccination_rate.csv'), index=False)

results.head(3)

 83%|████████▎ | 200/242 [2:08:59<27:05, 38.70s/it]   Process ForkPoolWorker-41:



KeyboardInterrupt: 

Process ForkPoolWorker-35:
Process ForkPoolWorker-39:
Process ForkPoolWorker-40:
Process ForkPoolWorker-33:
Process ForkPoolWorker-29:
Process ForkPoolWorker-37:
Process ForkPoolWorker-36:
Process ForkPoolWorker-38:
Process ForkPoolWorker-30:
Process ForkPoolWorker-31:
Process ForkPoolWorker-32:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-34:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jana/anaconda3/envs/covid/lib/python3.8/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jana/anaconda3/envs/covid/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jana/anaconda3/envs/covid/lib/python3.8/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "